In [ ]:
import random
import json

a = {
    "a": [1, 2, 3],
    "b": [4, 5, 6],
    "c": [7, 8, 9]
}


for key, value in a.items():
    random.shuffle(value)
    print(f"{key}: {value}")

for key, value in a.items():
    random.shuffle(value)
    print(f"{key}: {value}")

print(json.dumps(a, indent=2))

In [ ]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] * 11
import math
step = math.ceil(len(a) / 10)

print(f"Total versions: {len(a)}")
for group_index in range(10):
    test_versions = a[group_index * step:(group_index + 1) * step]
    print(f"Group {group_index + 1}: {len(test_versions)} versions")

In [ ]:
(15*9) + 8

In [ ]:
import os
import csv

curr_path = os.getcwd()
root_path = os.path.dirname(curr_path)

def save_mutation_info():
        full_MUTATION_MATRIX_CSV = f"{root_path}/full_mutation_matrix.csv"
        if not os.path.exists(full_MUTATION_MATRIX_CSV):
            print(f"Full mutation matrix CSV file {full_MUTATION_MATRIX_CSV} does not exist.")
            return
        
        # Save mutation information to the database
        with open(full_MUTATION_MATRIX_CSV, 'r') as csvfile:
            reader = csv.reader(csvfile)
            # skip the header
            next(reader, None)
            for row in reader:
                # Assuming the CSV has columns: mutant_id,class,method,line,mutator,result_transition,exception_type_transition,exception_msg_transition,stacktrace_transition,status,num_tests_run
                # print(row)
                mutation_idx = row[0]
                class_name = row[1]
                method = row[2]
                line = row[3]
                mutator = row[4]
                result_transition = row[5]
                exception_type_transition = row[6]
                exception_msg_transition = row[7]
                stacktrace_transition = row[8]
                status = row[9]
                num_tests_run = row[10]

                values = [
                    mutation_idx, class_name, method, line,
                    mutator, result_transition, exception_type_transition, status,
                    exception_msg_transition, stacktrace_transition, num_tests_run
                ]
                # self.DB.insert(
                #     "d4j_mutation_info",
                #     "fault_idx, mutation_idx, class, method, line, mutator, result_transition, exception_type_transition, status, exception_msg_transition, stacktrace_transition, num_tests_run",
                #     values
                # )

        print(f"Data saved for subject , bug ID , experiment label .")

In [ ]:
save_mutation_info()

In [5]:
# change current working directory to the root path
# to enable relative imports
import os
curr_path = os.getcwd()
root_path = os.path.dirname(curr_path)
os.chdir(root_path)

import dotenv
dotenv.load_dotenv()

from lib.database import CRUD


# DB_HOST=tester3.kaist.ac.kr
# DB_PORT=15432
# DB_USER=yangheechan
# DB_PASSWORD=yang1234
# DB=yangheechan
db = CRUD(
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB"),
    slack_channel=os.getenv("SLACK_CHANNEL"),
    slack_token=os.getenv("SLACK_TOKEN")
)


In [6]:
# merge all bit sequences into a single string
# the index of the bit is 1 if the bit is 1, otherwise 0

# int_result = (int(result_transition, 2) | 
#                   int(exception_type_transition, 2) |
#                   int(exception_msg_transition, 2) | 
#                   int(stacktrace_transition, 2))
#     return format(int_result, f'0{len(result_transition)}b')

def cov_bit_seq(tcs_results):
    bit_val = 0
    for item in tcs_results:
        # print(item)
        bit_seq_str = item[0]
        bit_seq = int(bit_seq_str, 2)  # Convert binary string to integer
        bit_val |= bit_seq  # Use bitwise OR to merge sequences
    return format(bit_val, f'0{len(tcs_results[0][0])}b')  # Convert back to binary string

# PASSING TEST CASES
passing_tcs = db.read(
    "d4j_tc_info",
    "line_coverage_bit_sequence",
    conditions = {
        "fault_idx": 159,
        "result": 0,
    }
)
print(f"Number of passing test cases: {len(passing_tcs)}")

bit_length = len(passing_tcs[0][0])

passing_tc_covered_bit_seq_str = cov_bit_seq(passing_tcs)

# FAILING TEST CASES
failing_tcs = db.read(
    "d4j_tc_info",
    "line_coverage_bit_sequence",
    conditions = {
        "fault_idx": 159,
        "result": 1,
    }
)
print(f"Number of failing test cases: {len(failing_tcs)}")

failing_tc_covered_bit_seq_str = cov_bit_seq(failing_tcs)

# Lets find the unique lines covered by failing tcs
bit_sequences = []
# failing_tc_covered_bit_seq is the lines covered by failing TCS (vise versa for passing TCS)
# What is the bitwise operation to find the unique lines covered by failing TCS? 
unique_failing_lines = int(failing_tc_covered_bit_seq_str, 2) & ~int(passing_tc_covered_bit_seq_str, 2)
unique_failing_lines_str = format(unique_failing_lines, f'0{bit_length}b')

ground_truth_line_idx = []
for i, bit in enumerate(unique_failing_lines_str):
    if bit == "1":
        print(f"Bit {i} is set to 1")
        ground_truth_line_idx.append(i)

Number of passing test cases: 2
Number of failing test cases: 1
Bit 6 is set to 1
Bit 7 is set to 1
Bit 18 is set to 1
Bit 19 is set to 1
Bit 20 is set to 1
Bit 21 is set to 1
Bit 22 is set to 1
Bit 23 is set to 1
Bit 24 is set to 1
Bit 25 is set to 1
Bit 26 is set to 1
Bit 27 is set to 1
Bit 34 is set to 1
Bit 36 is set to 1
Bit 37 is set to 1
Bit 38 is set to 1
Bit 39 is set to 1
Bit 40 is set to 1
Bit 41 is set to 1
Bit 42 is set to 1
Bit 78 is set to 1
Bit 79 is set to 1
Bit 80 is set to 1
Bit 81 is set to 1
Bit 82 is set to 1
Bit 83 is set to 1
Bit 84 is set to 1
Bit 85 is set to 1
Bit 86 is set to 1
Bit 87 is set to 1
Bit 88 is set to 1
Bit 89 is set to 1
Bit 95 is set to 1
Bit 105 is set to 1
Bit 106 is set to 1
Bit 107 is set to 1
Bit 108 is set to 1
Bit 109 is set to 1
Bit 110 is set to 1
Bit 111 is set to 1
Bit 112 is set to 1
Bit 113 is set to 1
Bit 114 is set to 1
Bit 115 is set to 1
Bit 116 is set to 1
Bit 117 is set to 1
Bit 118 is set to 1
Bit 119 is set to 1
Bit 120 is 

In [9]:
line_infos = db.read(
    "d4j_line_info",
    "line_idx, file, method, line_num",
    conditions={
        "fault_idx": 159,
    }
)

ground_truth_data = {}

# select 8 randomly from ground_truth_line_idx
import random
random.shuffle(ground_truth_line_idx)
ground_truth_line_idx = ground_truth_line_idx[:8]
for gt in ground_truth_line_idx:
    print(f"Ground truth line index: {gt}")

for line_info in line_infos:
    line_idx = line_info[0]
    file = line_info[1]
    method = line_info[2]
    line_num = line_info[3]

    if line_idx in ground_truth_line_idx:
        if file not in ground_truth_data:
            ground_truth_data[file] = {}
        if method not in ground_truth_data[file]:
            ground_truth_data[file][method] = []
        ground_truth_data[file][method].append((line_num, line_idx))

import json
print(json.dumps(ground_truth_data, indent=2))

Ground truth line index: 164
Ground truth line index: 211
Ground truth line index: 177
Ground truth line index: 193
Ground truth line index: 88
Ground truth line index: 262
Ground truth line index: 180
Ground truth line index: 83
{
  "org/apache/commons/math3/stat/ranking/TiesStrategy.java": {
    "<clinit>()": [
      [
        48,
        83
      ]
    ]
  },
  "org/apache/commons/math3/util/ArithmeticUtils.java": {
    "factorial(int)": [
      [
        306,
        88
      ]
    ]
  },
  "org/apache/commons/math3/util/FastMath.java": {
    "log(double,double[])": [
      [
        1321,
        164
      ]
    ],
    "pow(double,double)": [
      [
        1535,
        177
      ],
      [
        1538,
        180
      ],
      [
        1567,
        193
      ]
    ],
    "floor(double)": [
      [
        3334,
        211
      ]
    ]
  },
  "org/apache/commons/math3/util/ResizableDoubleArray.java": {
    "<init>(org.apache.commons.math3.util.ResizableDoubleArray)": [
  

In [10]:
pid = "Math"
bid = 30
gid = 0

for file, methods in ground_truth_data.items():
    print(f"File: {file}")
    for method, line_info in methods.items():
        for line_num, line_idx in line_info:
            # Assuming the description is "FAULT_OF_OMISSION"
            # You can change this to whatever description you want
            gid += 1
            value = [pid, bid, gid, file, method, line_num, line_idx, "FAULT_OF_OMISSION"]
            print(value)
            db.insert(
                "d4j_ground_truth_info",
                "pid, bid, gid, file, method, line, line_idx, description",
                value
            )

File: org/apache/commons/math3/stat/ranking/TiesStrategy.java
['Math', 30, 1, 'org/apache/commons/math3/stat/ranking/TiesStrategy.java', '<clinit>()', 48, 83, 'FAULT_OF_OMISSION']
File: org/apache/commons/math3/util/ArithmeticUtils.java
['Math', 30, 2, 'org/apache/commons/math3/util/ArithmeticUtils.java', 'factorial(int)', 306, 88, 'FAULT_OF_OMISSION']
File: org/apache/commons/math3/util/FastMath.java
['Math', 30, 3, 'org/apache/commons/math3/util/FastMath.java', 'log(double,double[])', 1321, 164, 'FAULT_OF_OMISSION']
['Math', 30, 4, 'org/apache/commons/math3/util/FastMath.java', 'pow(double,double)', 1535, 177, 'FAULT_OF_OMISSION']
['Math', 30, 5, 'org/apache/commons/math3/util/FastMath.java', 'pow(double,double)', 1538, 180, 'FAULT_OF_OMISSION']
['Math', 30, 6, 'org/apache/commons/math3/util/FastMath.java', 'pow(double,double)', 1567, 193, 'FAULT_OF_OMISSION']
['Math', 30, 7, 'org/apache/commons/math3/util/FastMath.java', 'floor(double)', 3334, 211, 'FAULT_OF_OMISSION']
File: org/apa

In [4]:
a = "00101"*100000
b = "01000"*100000
c = "11100"*100000
d = "00001"*100000

import time
def combine_transitions(*transitions):
    combined_transition = []
    for i in range(len(transitions[0])):
        combined_bit = (
            int(transitions[0][i]) |
            int(transitions[1][i]) |
            int(transitions[2][i]) |
            int(transitions[3][i])
        )
        combined_transition.append(str(combined_bit))
    return ''.join(combined_transition)

start_time = time.time()
combined1 = combine_transitions(a, b, c, d)
end_time = time.time()
print(f"Time taken to combine transitions: {end_time - start_time:.6f} seconds")

print(f"combined1 transition: {combined1}")

def combine_transitions_2(*transitions):
    int_result = (
        int(transitions[0], 2) |
        int(transitions[1], 2) |
        int(transitions[2], 2) |
        int(transitions[3], 2)
    )

    return format(int_result, f'0{len(transitions[0])}b')

start_time = time.time()
combined2 = combine_transitions_2(a, b, c, d)
end_time = time.time()
print(f"Time taken to combine transitions (2): {end_time - start_time:.6f} seconds")
print(f"Combined2 transition (2): {combined2}")

if combined1 == combined2:
    print("The combined transitions are equal.")
else:
    print("The combined transitions are not equal.")

Time taken to combine transitions: 0.226360 seconds
combined1 transition: ...
Time taken to combine transitions (2): 0.002666 seconds
Combined2 transition (2): ...
The combined transitions are equal.
